In [1]:
import cv2
import os 
from PIL import Image
import numpy as np

# Face Detection



In [2]:
# function for drawing rectangle 
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text):

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    coords = []

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        coords = [x, y, w, h]
    return coords


In [3]:
# loading haarcascades classifiers
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eyesCascade = cv2.CascadeClassifier('haarcascade_eye.xml')
noseCascade = cv2.CascadeClassifier('Nariz.xml')
mouthCascade = cv2.CascadeClassifier('Mouth.xml')

# function for features detection
def detect(img, faceCascade, eyeCascade, noseCascade, mouthCascade):
    color = {"blue":(255,0,0), "red":(0,0,255), "green":(0,255,0), "white":(255,255,255)}
    coords = draw_boundary(img, faceCascade, 1.1, 10, color['blue'], "Face")

    if len(coords)==4:
        roi_img = img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]
        coords = draw_boundary(roi_img, eyeCascade, 1.1, 12, color['red'], "Eye")
        coords = draw_boundary(roi_img, noseCascade, 1.1, 4, color['green'], "Nose")
        coords = draw_boundary(roi_img, mouthCascade, 1.1, 20, color['white'], "Mouth")
    return img


In [4]:
video_capture = cv2.VideoCapture(0)

# main detection part
while True:
    _, img = video_capture.read()
    img = detect(img, faceCascade, eyesCascade, noseCascade, mouthCascade)
    cv2.imshow("face detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


video_capture.release()
cv2.destroyAllWindows()

# Generating Datasets


In [25]:
# this will be called in data detection part
def generate_dataset(img, id, img_id):
    cv2.imwrite("data/Student."+str(id)+"."+str(img_id)+".jpg", img)
    

In [5]:
roll = int(input("Enter the roll number: "))

def detect(img, faceCascade, img_id, roll):
    
    color = {"blue":(255,0,0), "red":(0,0,255), "green":(0,255,0), "white":(255,255,255)}
    coords = draw_boundary(img, faceCascade, 1.1, 10, color['blue'], "Face")
   
    if len(coords)==4:
        # image crop to only detect face part
        roi_img = img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]
        user_id = roll
        generate_dataset(roi_img, user_id, img_id)

    return img


Enter the roll number: 48


In [42]:
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


video_capture = cv2.VideoCapture(0)

img_id = 0

while True:
    if img_id % 50 == 0:
        print("Collected ", img_id," images")

    _, img = video_capture.read()
    img = detect(img, faceCascade, img_id, roll)
    cv2.imshow("face detection", img)
    img_id += 1
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

Collected  0  images
Collected  50  images
Collected  100  images


# Training Images


In [20]:
def train_classifer(data_dir):
    # reads all images in data folder
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    # storing the images
    for image in path:
        img1 = Image.open(image).convert('L')
        img = img1.resize((186,170))
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    # Train and save classifier
    clf = cv2.face.EigenFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")


train_classifer("data")

# Recognize



In [6]:
# function for drawing rectangle 
def draw_boundary_predict(img, classifier, scaleFactor, minNeighbors, color, text, clf, rollnum):

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    coords = []

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        
        #Predicting part
        id, _ = clf.predict(gray_img[y:y+h, x:x+w])
        if id==rollnum:
            cv2.putText(img, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        coords = [x, y, w, h]
    return coords


In [7]:
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def recognize(img, clf, faceCascade):
    color = {"blue": (255, 0, 0), "red": (0, 0, 255), "green": (0, 255, 0), "white": (255, 255, 255)}
    coords = draw_boundary_predict(img, faceCascade, 1.1, 10, color["white"], "Face", clf, roll)
    return img




In [ ]:
clf = cv2.face.EigenFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    _, img1 = video_capture.read()
    img = img1.resize((186,170))
    img = recognize(img, clf, faceCascade)
    cv2.imshow("face detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()